# Authors' scholar impacts evaluation
The algorithms included in this evaluation are Total Publications(TP), Total Citations(TC), Weighted Citation(CS) and Leader Rank(LR).


## 1.Load Web of Science Bibilometric Data

In [1]:
from __future__ import division
import pandas as pd
import itertools
import numpy as np

PRecords = pd.read_csv('D:/_Research/Project_Sharing_Data_FromLinux/ProjectRebuild_2017/Data/Full_Record_WoS.csv', sep = ',', encoding = 'utf-8')
PRecords = PRecords.drop('Unnamed: 0', axis = 1)
PRecords

,AB,AF,AR,AU,BE,BN,BP,BS,C1,CA,...,SU,TC,TI,U1,U2,UT,VL,VR,WC,Z9
0,"In this study, net surface radiation (R-n) was...","Mahalakshmi, D. V.|Paul, Arati|Dutta, D.|Ali, ...",NaN,"Mahalakshmi, DV|Paul, A|Dutta, D|Ali, MM|Dadhw...",NaN,NaN,1,NaN,"[Mahalakshmi, D. V.; Ali, M. M.; Dadhwal, V. K...",NaN,...,NaN,0,Estimation of net surface radiation using eddy...,1,1,WOS:000381162400001,33,1.0,Geochemistry & Geophysics,0
1,"To date, direct validation of city-wide emissi...","Vaughan, Adam R.|Lee, James D.|Misztal, Pawel ...",NaN,"Vaughan, AR|Lee, JD|Misztal, PK|Metzger, S|Sha...",NaN,NaN,455,NaN,"[Vaughan, Adam R.] Univ York, Dept Chem, York,...",NaN,...,NaN,3,Spatially resolved flux measurements of NOx fr...,8,10,WOS:000380099700022,189,NaN,"Chemistry, Physical",3
2,Large variability in N2O emissions from manage...,"Grant, Robert F.|Neftel, Albrecht|Calanca, Pie...",NaN,"Grant, RF|Neftel, A|Calanca, P",NaN,NaN,3549,NaN,"[Grant, Robert F.] Univ Alberta, Dept Renewabl...",NaN,...,NaN,0,Ecological controls on N2O emission in surface...,11,12,WOS:000379427700003,13,NaN,"Ecology; Geosciences, Multidisciplinary",0
3,"Conversions of natural ecosystems, e.g., from ...","Merten, Jennifer|Roell, Alexander|Guillaume, T...",5,"Merten, J|Roll, A|Guillaume, T|Meijide, A|Tari...",NaN,NaN,NaN,NaN,"[Merten, Jennifer; Dittrich, Christoph; Faust,...",NaN,...,NaN,2,Water scarcity and oil palm expansion: social ...,16,28,WOS:000380049100006,21,NaN,Ecology; Environmental Studies,2
4,A scheme describing the process of stream-aqui...,"Zeng, Yujin|Xie, Zhenghui|Yu, Yan|Liu, Shuang|...",NaN,"Zeng, YJ|Xie, ZH|Yu, Y|Liu, S|Wang, LY|Jia, BH...",NaN,NaN,2333,NaN,"[Zeng, Yujin; Xie, Zhenghui; Liu, Shuang; Wang...",NaN,...,NaN,3,Ecohydrological effects of stream-aquifer wate...,10,15,WOS:000379419500013,20,NaN,"Geosciences, Multidisciplinary; Water Resources",3
5,There have been few studies conducted on the c...,"Yang, Zesu|Zhang, Qiang|Hao, Xiaocui",6809749,"Yang, ZS|Zhang, Q|Hao, XC",NaN,NaN,NaN,NaN,"[Yang, Zesu] Chengdu Univ Informat Technol, Co...",NaN,...,NaN,0,Evapotranspiration Trend and Its Relationship ...,8,8,WOS:000379433600001,NaN,NaN,Meteorology & Atmospheric Sciences,0
6,The lifetime of nitrogen oxides (NOx) affects ...,"Romer, Paul S.|Duffey, Kaitlin C.|Wooldridge, ...",NaN,"Romer, PS|Duffey, KC|Wooldridge, PJ|Allen, HM|...",NaN,NaN,7623,NaN,"[Romer, Paul S.; Duffey, Kaitlin C.; Wooldridg...",NaN,...,NaN,2,The lifetime of nitrogen oxides in an isoprene...,16,26,WOS:000379417300009,16,NaN,Meteorology & Atmospheric Sciences,2
7,"The emission, dispersion, and photochemistry o...","Su, Luping|Patton, Edward G.|de Arellano, Jord...",NaN,"Su, LP|Patton, EG|de Arellano, JVG|Guenther, A...",NaN,NaN,7725,NaN,"[Su, Luping; Mak, John E.] SUNY Stony Brook, S...",NaN,...,NaN,3,Understanding isoprene photooxidation using ob...,7,10,WOS:000379417300016,16,NaN,Meteorology & Atmospheric Sciences,3
8,"We measured volatile organic compounds (VOCs),...","Rantala, Pekka|Jarvi, Leena|Taipale, Risto|Lau...",NaN,"Rantala, P|Jarvi, L|Taipale, R|Laurila, TK|Pat...",NaN,NaN,7981,NaN,"[Rantala, Pekka; Jarvi, Leena; Taipale, Risto;...",NaN,...,NaN,0,Anthropogenic and biogenic influence on VOC fl...,3,12,WOS:000379417300032,16,NaN,Meteorology & Atmospheric Sciences,0
9,The dry component of total nitrogen and sulfur...,"Rumsey, Ian C.|Walker, John T.",NaN,"Rumsey, IC|Walker, JT",NaN,NaN,2581,NaN,"[Rumsey, Ian C.] Coll Charleston, Dept Phys & ...",NaN,...,NaN,0,Application of an online ion-chromatography-ba...,4,10,WOS:000379397100008,9,NaN,Meteorology & Atmospheric Sciences,0


## 2. Create citation relationship between papers
For further CiteScore and Leader Rank calculation, we must calculatin the inner citation among all recorded papers. In this case, we'll compute them using DI(DOI) and CR(Cited Refernces) fields.

In [8]:
PRecords.CR[0]

u'Al-Naimi Rasheed H., 2014, International Journal of Energy and Environment, V5|Augustine JA, 2000, B AM METEOROL SOC, V81, P2341, DOI 10.1175/1520-0477(2000)081<2341:SANSRB>2.3.CO;2|BALDOCCHI D, 2001, B AM METEOROL SOC, V82, P2415, DOI 10.1175/1520-0477(2001)082<2415:FANTTS>2.3.CO;2|Bandyopadhyay A, 2008, AGR FOREST METEOROL, V148, P1707, DOI 10.1016/j.agrformet.2008.06.002|Bocco M, 2010, CHIL J AGR RES, V70, P428, DOI 10.4067/S0718-58392010000300010|BRUTSAERT W, 1975, WATER RESOUR RES, V11, P742, DOI 10.1029/WR011i005p00742|Caner M, 2011, EXPERT SYST APPL, V38, P1668, DOI 10.1016/j.eswa.2010.07.090|Chaudhuri A. B., 1994, MANGROVES SUNDARBANS, V1, P1|Chen RS, 2006, ENERG CONVERS MANAGE, V47, P2991, DOI 10.1016/j.enconman.2006.03.025|Diak GR, 2000, AGR FOREST METEOROL, V103, P349, DOI 10.1016/S0168-1923(00)00141-6|Ertekin C, 2007, AGR FOREST METEOROL, V145, P36, DOI 10.1016/j.agrformet.2007.04.004|Fadare DA, 2009, APPL ENERG, V86, P1410, DOI 10.1016/j.apenergy.2008.12.005|Ferreira AG,

In [3]:
#Cited By Calculation
BeCitedBy = ['' for Cols in range(len(PRecords))]


def is_nan(obj): #Judge NaN
    if obj != obj:
        return(1)
    else:
        return(0)

for index, paper in PRecords.iterrows():
    DOI = paper['DI']
    if is_nan(DOI): #In case DOI is nan
        continue
    for sindex, spaper in PRecords.iterrows():
        if is_nan(spaper['CR']):
            continue
        if sindex < index:
            if DOI in spaper['CR']:
                BeCitedBy[index] += str(sindex) + '|'
        else:
            print(index) #Print the progress now
            break
PRecords['Be_Cited_By'] = BeCitedBy

0
1
2
3
4
5
6
7
8
9
10
11
12
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
43
44
50
51
53
54
55
56
57
58
59
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121


KeyboardInterrupt: 

this inner citation relationships between our papers consist a big directed graph of citation. To depict the graph, we'll create a matrix.

In [ ]:
def MakeGraph(Cols, Rows, Initial):
    return([[Initial for col in range(Cols)] for row in range(Rows)])

Cite_Relation = MakeGraph(len(PRecords), len(PRecords), 0)

#Initialize the Directed Graph
for index, paper in PRecords.iterrows():
    for cited in paper['Be_Cited_By'].split('|'):
        if cited != '':
            Cite_Relation[index][int(cited)] =  1

## 3. Calculate citation score of each author and cooperation replationships between authors
Both citation socres and relationships require unified authors, previously calculated authors will be loaded. Names and scores are attached.

In [2]:
UniAuthors = pd.read_csv('D:/_Research/Project_Sharing_Data_FromLinux/ProjectRebuild_2017/Data/Full_Author_Records.csv', sep = ',', encoding = 'utf-8')
UniAuthors = UniAuthors.drop('Unnamed: 0', axis = 1)
UniAuthors

,Name,POI,Paper,Short,Universal_Name_Tag,isFirst,isReprint
0,ARCADO TE,5603,10.1016/1352-2310(95)00136-M,"ARCADO, TE",0,0,0
1,Aalto Juho,751,NaN,"Aalto, J",1,0,0
2,Aalto P,2253,10.1016/j.atmosenv.2011.04.022,"Aalto, P",2,0,0
3,Aalto P,3041,10.1016/j.atmosenv.2009.02.049,"Aalto, P",2,0,0
4,Aalto P P,3180,NaN,"Aalto, PP",2,0,0
5,Aalto P,4900,10.1175/1520-0426(2004)021<0933:REASFS>2.0.CO;2,"Aalto, P",2,0,0
6,Aalto P,4913,NaN,"Aalto, P",2,0,0
7,Aalto P,5001,10.1029/2003JD003542,"Aalto, P",2,0,0
8,Aalto PP,5253,10.1029/2000JD900747,"Aalto, PP",2,0,0
9,Aalto T,2044,10.3402/tellusb.v64i0.18818,"Aalto, T",3,0,0


In [5]:
#Bring Names to index Authors
Names = []
SNames = []
temp = 0
nametemp = ''
snametemp = ''
for index, item in UniAuthors.iterrows():
    if temp == item.Universal_Name_Tag:
        nametemp += item.Name + '|'
        snametemp += item.Short + '|'
    else:
        Names.append(nametemp)
        SNames.append(snametemp)
        temp = item.Universal_Name_Tag
        nametemp = item.Name + '|'
        snametemp = item.Short + '|'
#Last name attach
Names.append(nametemp)
SNames.append(snametemp)

#Create Nameindex
NameIndex = []
for item in Names:
    temp = item.split('|')
    Lname = ''
    for name in temp:
        if len(name) > len(Lname):
            Lname = name
    NameIndex.append(Lname)

Total_pubs = [0 for Cols in range(UniAuthors.Universal_Name_Tag.iloc[-1] + 1)]
Total_citations = [0 for Cols in range(UniAuthors.Universal_Name_Tag.iloc[-1] + 1)]
Citation_scores = [float(0) for Cols in range(UniAuthors.Universal_Name_Tag.iloc[-1] + 1)]

#Total Citations and Citation Score calculation
for index, author in UniAuthors.iterrows():
    Total_pubs[author.Universal_Name_Tag] += 1
    Total_citations[author.Universal_Name_Tag] += PRecords.TC[author.POI]
    
    coauthor = len(PRecords.AF[author.POI].split('|'))
    if author.isReprint or author.isFirst:
        Citation_scores[author.Universal_Name_Tag] += PRecords.TC[author.POI]
    else:
        Citation_scores[author.Universal_Name_Tag] += PRecords.TC[author.POI] / coauthor

#Bind Total Citations and Citation Scores, Ouput
Scores = pd.DataFrame.from_dict({'Total_Publication':Total_pubs, 'Total_Citation':Total_citations, 'Citation_Score':Citation_scores, 'Names':Names, 'Short_Names':SNames, 'NameIndex':NameIndex})
Scores

,Citation_Score,NameIndex,Names,Short_Names,Total_Citation,Total_Publication
0,2.000000,ARCADO TE,ARCADO TE |,"ARCADO, TE|",32,1
1,1.000000,Aalto Juho,Aalto Juho |,"Aalto, J|",7,1
2,31.529762,Aalto P P,Aalto P |Aalto P |Aalto P P|Aalto P |Aalto P |...,"Aalto, P|Aalto, P|Aalto, PP|Aalto, P|Aalto, P|...",241,7
3,34.042857,Aalto Tuula,Aalto T |Aalto T |Aalto T |Aalto T |Aalto T |A...,"Aalto, T|Aalto, T|Aalto, T|Aalto, T|Aalto, T|A...",171,12
4,2.833333,Abaimov A P,Abaimov A P|,"Abaimov, AP|",17,1
5,0.101695,Abaoui J,Abaoui J |,"Abaoui, J|",6,1
6,0.000000,Abdalati Waleed,Abdalati Waleed |,"Abdalati, W|",0,1
7,5.581395,Abdalla Mohamed,Abdalla M |Abdalla M |Abdalla Mohamed |,"Abdalla, M|Abdalla, M|Abdalla, M|",156,3
8,0.000000,Abdalla Seifeldin H,Abdalla Seifeldin H|,"Abdalla, SH|",0,1
9,6.000000,Abdelghani Chehbouni,Abdelghani Chehbouni |,"Abdelghani, C|",6,1


## 3. LeaderRank Calculation
Here, every dataset required for calculate LearderRank scores are ready, a matrix which represents the relationship graph between authors will be built.
First we'll define all functions needed in this algorithm: 
1) MakeGraph, this function create a Cols X Rows matrix with initial value;
2) MakeCooperation, this function calculated the contribution value of cooperation for authors among each paper. Prvalue = 1 / (Count * (Count - 1)), here the Count is number of coauthors. These values will be aggregated and act as weights of graph connections;
3) InitialGraph, this function will add a new background node to the graph as described in LeaderRank algorithm []. Bidirectional connections are created between background nodes and every author, the connections from author to background nodes are weighted by Citation_scores;
4) MakeStochastic, this function brings the graph to random neutural status, waiting for iteration;

In [6]:
def MakeGraph(Cols, Rows, Initial):
    return [[Initial for col in range(Cols)] for row in range(Rows)]

def MakeCooperation(Graph, AuList):
    Count = len(AuList)
    if Count > 1:
        Prvalue = 1 / (Count * (Count - 1))
        Pairs = [itr for itr in itertools.permutations(AuList, 2)]
        for pair in Pairs:
            Graph[pair[0]][pair[1]] += Prvalue
    return Graph

def InitialGraph(Graph):
    for i in range(len(Graph)):
        Graph[0][i] = Citation_scores[i]
        Graph[i][0] = 1
    return Graph
        
def MakeStochastic(Graph):
    for i in range(len(Graph)):
        k_out = 1 / sum(Graph[i])
        for j in range(len(Graph)):
            Graph[i][j] = Graph[i][j] * k_out
    return Graph

Intialize the graph with both cooperation relationship and citation scores:

In [7]:
num_Authors = len(Scores)
CopGraph = MakeGraph(num_Authors, num_Authors, 0)
UniAuthors = UniAuthors.sort_values('POI')
temp = 0
AuList = []
for index, record in UniAuthors.iterrows():
    if temp == record.POI:
        AuList.append(record.Universal_Name_Tag)
    else:
        CopGraph = MakeCooperation(CopGraph, AuList)
        temp = record.POI
        AuList = [record.Universal_Name_Tag]
        print 'Current processing Paper:', record.POI

CopGraph = InitialGraph(CopGraph)
CopGraph = MakeStochastic(CopGraph)

Current processing Paper: 1
Current processing Paper: 2
Current processing Paper: 3
Current processing Paper: 4
Current processing Paper: 5
Current processing Paper: 6
Current processing Paper: 7
Current processing Paper: 8
Current processing Paper: 9
Current processing Paper: 10
Current processing Paper: 11
Current processing Paper: 12
Current processing Paper: 13
Current processing Paper: 14
Current processing Paper: 15
Current processing Paper: 16
Current processing Paper: 17
Current processing Paper: 18
Current processing Paper: 19
Current processing Paper: 20
Current processing Paper: 21
Current processing Paper: 22
Current processing Paper: 23
Current processing Paper: 24
Current processing Paper: 25
Current processing Paper: 26
Current processing Paper: 27
Current processing Paper: 28
Current processing Paper: 29
Current processing Paper: 30
Current processing Paper: 31
Current processing Paper: 32
Current processing Paper: 33
Current processing Paper: 34
Current processing Pape

Iterate this graph with LeaderRank Algorithm until it reaches the threshold. Average background node score and distribute it each author nodes.

In [8]:
LeaderRank = [1 for cols in range(num_Authors)]
LeaderRank[0] = 0
error = 10000
error_threshold = 0.0002
step = 1

CopGraph = np.mat(CopGraph)
LeaderRank = np.mat(LeaderRank)

while error > error_threshold:
    temp = LeaderRank
    LeaderRank = LeaderRank * CopGraph
    error = abs(LeaderRank - temp).sum() / temp.sum()
    print 'Current Iteration:', step, 'Current Error:', error
    step += 1

Avg = LeaderRank[0,0] / num_Authors
LeaderRank += Avg
LeaderRank[0,0] = Avg

Current Iteration: 1 Current Error: 1.53130143895
Current Iteration: 2 Current Error: 1.26700817836
Current Iteration: 3 Current Error: 0.558009403501
Current Iteration: 4 Current Error: 0.218373396699
Current Iteration: 5 Current Error: 0.0793031329539
Current Iteration: 6 Current Error: 0.0276841204274
Current Iteration: 7 Current Error: 0.00923526802375
Current Iteration: 8 Current Error: 0.00318438028981
Current Iteration: 9 Current Error: 0.00108427319233
Current Iteration: 10 Current Error: 0.000447075155001
Current Iteration: 11 Current Error: 0.000222604966598
Current Iteration: 12 Current Error: 0.000107380517242


LeaderRank calculation complete, attach this attribute to Scores table, output them to local disk.

In [9]:
Scores['LeaderRank'] = LeaderRank.transpose()
Scores.to_csv('D:/_Research/Project_Sharing_Data_FromLinux/ProjectRebuild_2017/Data/Universal_Author_Scores.csv')

In [10]:
Scores

,Citation_Score,NameIndex,Names,Short_Names,Total_Citation,Total_Publication,LeaderRank
0,2.000000,ARCADO TE,ARCADO TE |,"ARCADO, TE|",32,1,0.281736
1,1.000000,Aalto Juho,Aalto Juho |,"Aalto, J|",7,1,0.406272
2,31.529762,Aalto P P,Aalto P |Aalto P |Aalto P P|Aalto P |Aalto P |...,"Aalto, P|Aalto, P|Aalto, PP|Aalto, P|Aalto, P|...",241,7,1.863562
3,34.042857,Aalto Tuula,Aalto T |Aalto T |Aalto T |Aalto T |Aalto T |A...,"Aalto, T|Aalto, T|Aalto, T|Aalto, T|Aalto, T|A...",171,12,1.864732
4,2.833333,Abaimov A P,Abaimov A P|,"Abaimov, AP|",17,1,0.367166
5,0.101695,Abaoui J,Abaoui J |,"Abaoui, J|",6,1,0.284561
6,0.000000,Abdalati Waleed,Abdalati Waleed |,"Abdalati, W|",0,1,0.388332
7,5.581395,Abdalla Mohamed,Abdalla M |Abdalla M |Abdalla Mohamed |,"Abdalla, M|Abdalla, M|Abdalla, M|",156,3,0.425961
8,0.000000,Abdalla Seifeldin H,Abdalla Seifeldin H|,"Abdalla, SH|",0,1,0.596599
9,6.000000,Abdelghani Chehbouni,Abdelghani Chehbouni |,"Abdelghani, C|",6,1,0.395203
